In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from xml.dom import minidom
from Layer import Layer

## Try other parser

In [5]:
import xml.etree.ElementTree as ET
file_path = "snowpits_200_MT/snowpits-66387-caaml.xml"

common_tag = '{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}'
root = ET.parse(file_path).getroot()


In [7]:
# measurementDirection
try:
    measurementDirection = next(root.iter(common_tag + 'SnowProfileMeasurements'), None).get('dir')
except AttributeError:
    measurementDirection = None

print(measurementDirection)


top down


In [5]:
hs_tag = common_tag + 'profileDepth'
        
try:
    hs = next(root.iter(hs_tag), None).text
    hs_units = next(root.iter(hs_tag), None).get('uom')
    HS = [float(hs), hs_units]
except AttributeError:
    HS = None


Surface Conditions
windLoading
penetrationFoot
penetrationSki

In [21]:
custom = root.iter(common_tag+'custom_Data')

for tag in custom:
    print(tag.text)

In [ ]:
try:
    penFoot_val = next(root.iter(common_tag + 'penetrationFoot'), None).text
    penFoot_units = next(root.iter(common_tag + 'penetrationFoot'), None).get('uom')
    penFoot = [float(penFoot_val), penFoot_units]
except AttributeError:
    penFoot = None

In [31]:
# layers
layers = root.iter(common_tag + 'Layer')

for layer in layers:
    for prop in layer.iter():
        if prop.tag.endswith('depthTop'):
            depthTop_val = prop.text
            depthTop_units = prop.get('uom')
            depthTop=[depthTop_val,depthTop_units]
            print(depthTop)

        

['0', 'cm']
['8', 'cm']
['47', 'cm']
['48', 'cm']
['23', 'cm']


In [6]:
# pitID
#  <caaml:locRef gml:id="location-nid-66387">

pitID_tag = common_tag + 'locRef'
gml_tag = '{http://www.opengis.net/gml}id'

try:
    pitID_str = next(root.iter(pitID_tag), None).attrib[gml_tag]
    pitID = pitID_str.split('-')[-1]
except AttributeError:
    pitID = None

print(pitID)

66387
